# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [3]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [5]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [6]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/france-cedric-jubillar-condamne-a-30-ans-de-reclusion-pour-le-meurtre-de-son-epouse-11618044',
  'Cédric Jubillar condamné à 30 ans de réclusion pour le meurtre de son épouse'],
 ['https://www.rtbf.be/article/que-vaut-un-passeport-belge-aujourd-hui-et-surtout-que-cache-le-prix-du-privilege-de-franchir-les-frontieres-11617767',
  'Que vaut un passeport belge aujourd’hui\xa0? Et surtout, que cache le "prix" du privilège de franchir les frontières\xa0?'],
 ['https://www.rtbf.be/article/toxicite-des-pesticides-la-societe-scientifique-de-medecine-generaliste-denonce-les-faux-d-alfred-bernard-11617322',
  "Toxicité des pesticides : la Société Scientifique de Médecine Généraliste dénonce le discours d'Alfred Bernard"],
 ['https://www.rtbf.be/article/wrc-europe-centrale-six-speciales-au-programme-ce-vendredi-direct-commente-a-8h30-11617632',
  'WRC Europe Centrale : duel haletant entre Ogier et Rovanpera, qui revient à 2.4 secondes (direct vidéo à 17h20)'],
 ['ht

## Création d'un dataframe avec les liens et les titres des articles


In [7]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/france-cedric-jubi...,Cédric Jubillar condamné à 30 ans de réclusion...
1,https://www.rtbf.be/article/que-vaut-un-passep...,Que vaut un passeport belge aujourd’hui ? Et s...
2,https://www.rtbf.be/article/toxicite-des-pesti...,Toxicité des pesticides : la Société Scientifi...
3,https://www.rtbf.be/article/wrc-europe-central...,WRC Europe Centrale : duel haletant entre Ogie...
4,https://www.rtbf.be/article/rencontre-avec-un-...,"Rencontre avec un magnétiseur, un métier qui s..."
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,Noël en héritage
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Feux de l'amour S27
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Un parfum de Noël
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Coup de foudre glacé


In [8]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [9]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

Europe
Le peintre-plaquiste de 38 ans a regardé impassible la présidente énoncer la décision, mains serrées sur l'ouverture vitrée du box. Le verdict est conforme aux réquisitions du ministère public.
Les avocats de Cédric Jubillar, qui réclamaient l'acquittement, ont annoncé leur intention de faire appel de cette condamnation. Un nouveau procès devrait se tenir en 2026, probablement devant la cour d'appel de Toulouse.
Dans la matinée, pour sa dernière prise de parole, l'accusé avait une nouvelle fois assuré n'avoir " absolument rien fait à Delphine ", infirmière disparue en décembre 2020 dans le sud-ouest de la France, à 33 ans. Son corps n'a jamais été retrouvé.
Après quatre semaines de procès, la cour d'assises du Tarn, composée de trois magistrats et de six jurés populaires, a délibéré pendant près de cinq heures, pour répondre à la question : Cédric Jubillar " est-il coupable d'avoir, dans la nuit du 15 au 16 décembre 2020 à Cagnac-les-Mines donné volontairement la mort à Delphine

### Nettoyage du texte à l'aide d'expressions régulières

In [10]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Le peintre-plaquiste de 38 ans a regardé impassible la présidente énoncer la décision, mains serrées sur l'ouverture vitrée du box. Le verdict est conforme aux réquisitions du ministère public.
Les avocats de Cédric Jubillar, qui réclamaient l'acquittement, ont annoncé leur intention de faire appel de cette condamnation. Un nouveau procès devrait se tenir en 2026, probablement devant la cour d'appel de Toulouse.
Dans la matinée, pour sa dernière prise de parole, l'accusé avait une nouvelle fois assuré n'avoir " absolument rien fait à Delphine ", infirmière disparue en décembre 2020 dans le sud-ouest de la France, à 33 ans. Son corps n'a jamais été retrouvé.
Après quatre semaines de procès, la cour d'assises du Tarn, composée de trois magistrats et de six jurés populaires, a délibéré pendant près de cinq heures, pour répondre à la question : Cédric Jubillar " est-il coupable d'avoir, dans la nuit du 15 au 16 décembre 2020 à Cagnac-les-Mines donné volontairement la mort à Delphine Aussag

### Création d'un fichier avec le contenu de l'article


In [12]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp